# Interactive 3D Animated Visualization of ASL using Plotly! 

The aim of this notebook is to provide a powerful tool for visualizing and analyzing 3D ASL using the data provided in the competition. The notebook consists of several helper functions and one main function that is responsible to generate a 3D visualization using Plotly. **The function only takes as input any of the parquet data (converted to a dataframe)** containing the x, y, z coordinates of each landmark.

This will allow for a more intuitive understanding of the data provided for augmentation, as it shows the hands, pose and face landmarks from different angles and allows for rotation and zooming. This is particularly important for ASL recognition, as hand shape and orientation are key components of sign language.

Additionally, the 3D visualization can be used for data exploration and analysis. The ability to rotate and zoom in on the hand can reveal patterns and relationships in the data that may not be apparent in a 2D representation. This can help you kagglers better understand the characteristics of different poses and improve our ASL recognition models.

Sneak peak 👀👇

<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExY2FkMzA2OWYxZTA3MzQ4OGQ1ODIyYWRlZmE4ZDBhYjVlMTZkNmQ0OCZjdD1n/IDMyhLF7HMqEEk7eIB/giphy.gif">

In [1]:
# import the desired packages
import numpy as np
import pandas as pd
import json
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "simple_white"

## Helper Functions 

These two functions are called within the main function to assign specific colors to different landmark types as well and for correctly connecting landmarks between one another. 

In [2]:
# assign desired colors to landmarks
def assign_color(row):
    if row == 'face':
        return 'red'
    elif 'hand' in row:
        return 'dodgerblue'
    else:
        return 'green'

# specifies the plotting order
def assign_order(row):
    if row.type == 'face':
        return row.landmark_index + 101
    elif row.type == 'pose':
        return row.landmark_index + 30
    elif row.type == 'left_hand':
        return row.landmark_index + 80
    else:
        return row.landmark_index

## Main Function 

In [3]:
def visualise3d_landmarks(parquet_df):
    connections = [  # right hand
        [0, 1, 2, 3, 4, ],
        [0, 5, 6, 7, 8],
        [0, 9, 10, 11, 12],
        [0, 13, 14, 15, 16],
        [0, 17, 18, 19, 20],

        # pose
        [38, 36, 35, 34, 30, 31, 32, 33, 37],
        [40, 39],
        [52, 46, 50, 48, 46, 44, 42, 41, 43, 45, 47, 49, 45, 51],
        [42, 54, 56, 58, 60, 62, 58],
        [41, 53, 55, 57, 59, 61, 57],
        [54, 53],

        # left hand
        [80, 81, 82, 83, 84, ],
        [80, 85, 86, 87, 88],
        [80, 89, 90, 91, 92],
        [80, 93, 94, 95, 96],
        [80, 97, 98, 99, 100], ]

    frames = sorted(set(parquet_df.frame))
    first_frame = min(frames)
    parquet_df['color'] = parquet_df.type.apply(lambda row: assign_color(row))
    parquet_df['plot_order'] = parquet_df.apply(lambda row: assign_order(row), axis=1)
    first_frame_df = parquet_df[parquet_df.frame == first_frame].copy()
    first_frame_df = first_frame_df.sort_values(["plot_order"]).set_index('plot_order')


    frames_l = []
    for frame in frames:
        filtered_df = parquet_df[parquet_df.frame == frame].copy()
        filtered_df = filtered_df.sort_values(["plot_order"]).set_index("plot_order")
        traces = [go.Scatter3d(
            x=filtered_df['x'],
            y=filtered_df['y'],
            z=filtered_df['z'],
            mode='markers',
            marker=dict(
                color=filtered_df.color,
                size=3))]

        for i, seg in enumerate(connections):
            trace = go.Scatter3d(
                    x=filtered_df.loc[seg]['x'],
                    y=filtered_df.loc[seg]['y'],
                    z=filtered_df.loc[seg]['z'],
                    mode='lines',
            )
                # append the trace to the list of traces
            traces.append(trace)
        frame_data = go.Frame(data=traces, traces = [i for i in range(16)])
        # append the frame to the list of frames
        frames_l.append(frame_data)

    traces = [go.Scatter3d(
        x=first_frame_df['x'],
        y=first_frame_df['y'],
        z=first_frame_df['z'],
        mode='markers',
        marker=dict(
            color=first_frame_df.color,
            size=3
        )
    )]
    for i, seg in enumerate(connections):
        trace = go.Scatter3d(
            x=first_frame_df.loc[seg]['x'],
            y=first_frame_df.loc[seg]['y'],
            z=first_frame_df.loc[seg]['z'],
            mode='lines',
            line=dict(
                color='black',
                width=3
            )
        )
        # append the trace to the list of traces
        traces.append(trace)
    fig = go.Figure(
        data=traces,
        frames=frames_l
    )

    # Layout
    fig.update_layout(
        title="ASL Sign Visualization",
        width=1000,
        height=1000,
        scene={
            'aspectmode': 'data',
        },
        updatemenus=[{"buttons": [
                    {"args": [None, {"frame": {"duration": 100,"redraw": True},
                                    "fromcurrent": True,
                                    "transition": {"duration": 0}}],
                        "label": "&#9654;",  # play symbol
                        "method": "animate",},],
                "direction": "left",
                "pad": {"r": 100, "t": 100},
                "font": {"size":16},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }])
    
    camera = dict(
        up=dict(x=0, y=-1, z=0),
        eye=dict(x=0, y=0, z=2.5))
    
    fig.update_layout(scene_camera=camera, showlegend=False)
    fig.update_layout(scene = dict(xaxis = dict(visible=False),
            yaxis = dict(visible=False),
            zaxis =dict(visible=False, autorange = "reversed")))
    fig.show()

## DEMO

In [4]:
parquet_df = pd.read_parquet("/kaggle/input/asl-signs/train_landmark_files/26734/1000035562.parquet")
visualise3d_landmarks(parquet_df)
### YES ITS THAT EASY TO USE!!!
### You can rotate, zoom and move the 3d plot below as much as you want! dont forget to press the play button!

## 2D Version

In [5]:
def visualise2d_landmarks(parquet_df):
    connections = [  
        [0, 1, 2, 3, 4,],
        [0, 5, 6, 7, 8],
        [0, 9, 10, 11, 12],
        [0, 13, 14, 15, 16],
        [0, 17, 18, 19, 20],

        
        [38, 36, 35, 34, 30, 31, 32, 33, 37],
        [40, 39],
        [52, 46, 50, 48, 46, 44, 42, 41, 43, 45, 47, 49, 45, 51],
        [42, 54, 56, 58, 60, 62, 58],
        [41, 53, 55, 57, 59, 61, 57],
        [54, 53],

        
        [80, 81, 82, 83, 84, ],
        [80, 85, 86, 87, 88],
        [80, 89, 90, 91, 92],
        [80, 93, 94, 95, 96],
        [80, 97, 98, 99, 100], ]


    frames = sorted(set(parquet_df.frame))
    first_frame = min(frames)
    parquet_df['color'] = parquet_df.type.apply(lambda row: assign_color(row))
    parquet_df['plot_order'] = parquet_df.apply(lambda row: assign_order(row), axis=1)
    first_frame_df = parquet_df[parquet_df.frame == first_frame].copy()
    first_frame_df = first_frame_df.sort_values(["plot_order"]).set_index('plot_order')


    frames_l = []
    for frame in frames:
        filtered_df = parquet_df[parquet_df.frame == frame].copy()
        filtered_df = filtered_df.sort_values(["plot_order"]).set_index("plot_order")
        traces = [go.Scatter(
            x=filtered_df['x'],
            y=filtered_df['y'],
            mode='markers',
            marker=dict(
                color=filtered_df.color,
                size=9))]

        for i, seg in enumerate(connections):
            trace = go.Scatter(
                    x=filtered_df.loc[seg]['x'],
                    y=filtered_df.loc[seg]['y'],
                    mode='lines',
            )
            traces.append(trace)
        frame_data = go.Frame(data=traces, traces = [i for i in range(17)])
        frames_l.append(frame_data)

    traces = [go.Scatter(
        x=first_frame_df['x'],
        y=first_frame_df['y'],
        mode='markers',
        marker=dict(
            color=first_frame_df.color,
            size=9
        )
    )]
    for i, seg in enumerate(connections):
        trace = go.Scatter(
            x=first_frame_df.loc[seg]['x'],
            y=first_frame_df.loc[seg]['y'],
            mode='lines',
            line=dict(
                color='black',
                width=2
            )
        )
        traces.append(trace)
    fig = go.Figure(
        data=traces,
        frames=frames_l
    )


    fig.update_layout(
        title="ASL Sign Visualization",
        width=500,
        height=800,
        scene={
            'aspectmode': 'data',
        },
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 100,
                                                  "redraw": True},
                                        "fromcurrent": True,
                                        "transition": {"duration": 0}}],
                        "label": "&#9654;",
                        "method": "animate",
                    },

                ],
                "direction": "left",
                "pad": {"r": 100, "t": 100},
                "font": {"size":30},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
    )
    camera = dict(
        up=dict(x=0, y=-1, z=0),
        eye=dict(x=0, y=0, z=2.5)
    )
    fig.update_layout(scene_camera=camera, showlegend=False)
    fig.update_layout(xaxis = dict(visible=False),
            yaxis = dict(visible=False),
    )
    fig.update_yaxes(autorange="reversed")

    fig.show()

In [6]:
visualise2d_landmarks(parquet_df)